In [24]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as fun 
import numpy as np
from pyspark.sql.types import *

In [25]:
spark = SparkSession.builder.getOrCreate()

In [26]:
sc = spark.sparkContext

Create an RDD from a list of numbers (1,50) using numpy methods

In [27]:
nums = np.arange(1, 50)
rdd_nums = sc.parallelize(nums)
print(rdd_nums.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


Find the sum, average, maximum, minimum, and count

In [28]:
#Sum
print(rdd_nums.sum())

1225


In [29]:
#avg
print(rdd_nums.mean())

25.0


In [30]:
#Count
print(rdd_nums.count())

49


In [31]:
#Min
print(rdd_nums.min())

1


In [32]:
#Max
print(rdd_nums.max())

49


Count how many numbers are even vs. odd.

In [35]:
odd_nums = rdd_nums.filter(lambda x: x % 2 != 0)
even_nums = rdd_nums.filter(lambda x: x % 2 == 0)
print("Odd numbers:", odd_nums.count())
print("Even numbers:", even_nums.count())

Odd numbers: 25
Even numbers: 24


You have the following data of people info ('Name', 'Age'), answer the following questions

In [36]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
rdd_people.collect()

[('Nada ', 25),
 ('Mona', 30),
 ('Ahmed', 35),
 ('Khaled', 40),
 ('Ahmed', 35),
 ('Nada ', 25)]

Find the oldest person

In [41]:
oldest_person = rdd_people.max(lambda x: x[1])
print("Oldest person:", oldest_person)

Oldest person: ('Khaled', 40)


Compute the average age

In [42]:
avg_age = rdd_people.map(lambda x: x[1]).mean()
print("Average age:", avg_age)

Average age: 31.666666666666668


Group all the names by their age

In [48]:
grouped = rdd_people.groupBy(lambda x: x[1]).mapValues(lambda vals: [v[0] for v in vals])
print(grouped.collect())

[(25, ['Nada ', 'Nada ']), (40, ['Khaled']), (30, ['Mona']), (35, ['Ahmed', 'Ahmed'])]


Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"

In [54]:
rdd_russia = sc.textFile("/data/russia.txt")
rdd_russia.collect()

['"Russia is the largest country in the world by land area',
 'Moscow is the capital city of Russia',
 'The Russian language is one of the most widely spoken languages in the world',
 'Russia is known for its rich history and culture',
 'The Trans-Siberian Railway is the longest railway line in the world',
 'Russia has a strong tradition in literature, music and ballet',
 'The country is famous for its cold winters and vast landscapes',
 'Russia is a major player in global energy production"']

Count the total number of lines.

In [56]:
total_lines = rdd_russia.count()
print("Total lines in russia.txt:", total_lines)

Total lines in russia.txt: 8


Count how many lines contain the word "Russia"

In [ ]:
lines_contain_russia = rdd_russia.filter(lambda line: "Russia" in line).count()
print("Lines containing 'Russia':", lines_contain_russia)

Lines containing 'Russia': 6


Find the most 5 frequent word in the file.

In [64]:
word_counts = rdd_russia.flatMap(lambda line: line.split()).countByValue()
most_5_frequent_words = sorted(word_counts.items(), key=lambda x: -x[1])[:5]
print("5 most frequent words:", most_5_frequent_words)

5 most frequent words: [('is', 7), ('the', 7), ('in', 5), ('Russia', 4), ('world', 3)]


Tokenize words

In [72]:
words = rdd_russia.flatMap(lambda line: line.split())
words = words.map(lambda w: w.lower())
print("Total unique words:", words.collect())

Total unique words: ['"russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'moscow', 'is', 'the', 'capital', 'city', 'of', 'russia', 'the', 'russian', 'language', 'is', 'one', 'of', 'the', 'most', 'widely', 'spoken', 'languages', 'in', 'the', 'world', 'russia', 'is', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'the', 'trans-siberian', 'railway', 'is', 'the', 'longest', 'railway', 'line', 'in', 'the', 'world', 'russia', 'has', 'a', 'strong', 'tradition', 'in', 'literature,', 'music', 'and', 'ballet', 'the', 'country', 'is', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'russia', 'is', 'a', 'major', 'player', 'in', 'global', 'energy', 'production"']


Remove stopwords (a, the, is, to, in, of). 

In [73]:
stopwords = {"a", "the", "is", "to", "in", "of"}
filtered_words = words.filter(lambda w: w not in stopwords)
print("Filtered words:", filtered_words.collect())

Filtered words: ['"russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'moscow', 'capital', 'city', 'russia', 'russian', 'language', 'one', 'most', 'widely', 'spoken', 'languages', 'world', 'russia', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'trans-siberian', 'railway', 'longest', 'railway', 'line', 'world', 'russia', 'has', 'strong', 'tradition', 'literature,', 'music', 'and', 'ballet', 'country', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'russia', 'major', 'player', 'global', 'energy', 'production"']


Count the frequency of each word

In [74]:
words_frequency = filtered_words.map(lambda w: (w, 1)).reduceByKey(lambda a, b: a + b)
print("Words frequency:", words_frequency.collect())

Words frequency: [('"russia', 1), ('largest', 1), ('country', 2), ('world', 3), ('by', 1), ('land', 1), ('area', 1), ('capital', 1), ('russian', 1), ('language', 1), ('most', 1), ('widely', 1), ('known', 1), ('for', 2), ('history', 1), ('and', 3), ('line', 1), ('literature,', 1), ('music', 1), ('famous', 1), ('cold', 1), ('winters', 1), ('landscapes', 1), ('player', 1), ('energy', 1), ('production"', 1), ('moscow', 1), ('city', 1), ('russia', 4), ('one', 1), ('spoken', 1), ('languages', 1), ('its', 2), ('rich', 1), ('culture', 1), ('trans-siberian', 1), ('railway', 2), ('longest', 1), ('has', 1), ('strong', 1), ('tradition', 1), ('ballet', 1), ('vast', 1), ('major', 1), ('global', 1)]


In [75]:
schema = 'id integer, name string, age integer, salary integer' 
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [76]:
df.printSchema()
df.show(2)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [77]:
df.select("name", "salary").show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [78]:
df.select(fun.avg("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|     6000.0|
+-----------+



Filter employees older than 28

In [79]:
df.filter(df.age > 28).show()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column

In [ ]:
df.select("name").distinct().count()

4

Group by a the name column and find average salary

In [82]:
df.groupBy("name").agg(fun.avg("salary").alias("Average")).show()

+------+-------+
|  name|Average|
+------+-------+
|   Ali| 4000.0|
|Mariam| 6750.0|
|  Omar| 6750.0|
|  Sara| 5000.0|
+------+-------+



In [83]:
df1 = spark.read.csv("/data/NullData.csv", header=True, inferSchema=True) #this file in shared folder
df1.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Find the avg sales 

In [84]:
df1.select(fun.avg("Sales")).show()

+----------+
|avg(Sales)|
+----------+
|     400.5|
+----------+



Replace null name with 'Unknown' and sales with the avg sales of the column 

In [88]:
avg_sales_val = df1.agg(fun.avg("Sales")).first()[0]
df1_filled = df1.fillna({"name": "Unknown", "Sales": avg_sales_val})
df1_filled.show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John|400.5|
|emp2|Unknown|400.5|
|emp3|Unknown|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

